# All podcasts

### Load elements and build DataFrame

In [1]:
import pandas as pd
import re
import pickle
import nltk
import spacy
from spacy.lang.en import English
import collections
from collections import Counter
from itertools import chain
import statistics
import math

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# initialize spacy objects
nlp = spacy.load('en_core_web_md')

In [3]:
%store -r Nightvale_df
%store -r myDNA_df
%store -r YWA_df
%store -r uu_df
%store -r radiolab_df
%store -r tal_df
%store -r bullseye
%store -r mother
%store -r hodgman
%store -r flophouse
%store -r switchblade
%store -r mbmbam
%store -r sawbones
%store -r wonderful
%store -r tgg
%store -r ffire
%store -r shmanners
%store -r taz
%store -r neoscum_df
%store -r allusionist_df


# %store -r freak_df
# %store -r Lore_df
# %store -r Invisible_df
# %store -r OnBeing_df
# %store -r StoryCorps_df

In [4]:
radiolab_df.head(3)

,Title,Year,Text
0,Bit Flip,2019,[ADVERTISEMENT] [RADIOLAB INTRO] SIMON ...
1,Dispatches from 1918,2020,[RADIOLAB INTRO] PAT WALTERS: Jad? JAD ...
2,The Beauty Puzzle,2019,Radiolab: The Beauty Puzzle CHAPTER 1 (D...


In [5]:
# create a multindexed dataframe
data = pd.concat([Nightvale_df.reset_index(drop=True),
                  myDNA_df.reset_index(drop=True),
                  YWA_df.reset_index(drop=True),                  
                  uu_df.reset_index(drop=True),
                  radiolab_df.reset_index(drop=True),
                  tal_df.reset_index(drop=True),
                  bullseye,
                  mother,
                  hodgman,
                  flophouse,
                  switchblade,
                  mbmbam,
                  sawbones,
                  wonderful,
                  tgg,
                  ffire,
                  shmanners,
                  taz,
                  neoscum_df,
                  allusionist_df],
                  # StoryCorps_df, OnBeing_df, Invisible_df, Lore_df],
                  keys = ['Welcome to Nightvale','Move Your DNA','You\'re Wrong About','Unlocking Us',
                         'Radiolab','This American Life', 'Bullseye with Jesse Thorn','One Bad Mother',
                         'Judge John Hodgman','The Flophouse','Switchblade Sisters',
                         'MBMBaM','Sawbones','Wonderful','The Greatest Generation','Friendly Fire','Shmanners',
                          'The Adventure Zone','NeoScum', 'The Allusionist'], names=['podcast','#']).reset_index(level=1)

In [6]:
data.sample(10)

,#,Episode,Year,Title,Text,Anonymized_text,Speaker_parts,Podcast
podcast,,,,,,,,
Radiolab,162,NaN,2021,More Money Less Problems,ANNOUNCER: Listener-supported WNYC Studios. ...,NaN,NaN,NaN
Welcome to Nightvale,153,169,2020,the whittler,"Let us go then you and I, when the evening is...",NaN,NaN,NaN
This American Life,609,612,NaN,Ask a Grown-Up,"Prologue Ira Glass Well, it's Th...",NaN,NaN,NaN
Radiolab,251,NaN,2014,KILL 'EM ALL,"Speaker 1: Hey, wait, you're ... Speaker ...",NaN,NaN,NaN
MBMBaM,3,516,,radical gamer facts,MBMBaM 516: Radical Gamer Facts \nPublished ...,NaN,NaN,mbmbam
One Bad Mother,11,369,NaN,Non-traditional Parenting with If These Ovari...,biz ellis Hi. I’m Biz. theresa thorn And I’m T...,﻿ SPEAKER ellis hi. i’m SPEAKER. SPEAKER ...,{'caller': ['things are doing— . dammit. th...,one bad mother
Unlocking Us,2,NaN,2020,"Brené on The Queen's Gambit, Revisiting FFTs, ...","Transcript Brené Brown: Hi everyone, I’m Bre...",NaN,NaN,NaN
The Allusionist,65,058,2017,eclipse,visit theREDACTED.org/eclipse to read about ...,NaN,NaN,NaN
This American Life,124,125,NaN,Apocalypse,Prologue Ira Glass From WBEZ Chicag...,NaN,NaN,NaN


In [7]:
data = data.drop(columns=['#', 'Podcast'])
data.sample(5)

,Episode,Year,Title,Text,Anonymized_text,Speaker_parts
podcast,,,,,,
MBMBaM,529,,throw me in the dumpster behind the applebees ...,MBMBaM 529: Throw Me In the Dumpster Behind th...,NaN,NaN
This American Life,146,NaN,Urban Nature,Prologue Ira Glass Kate Aurthur be...,NaN,NaN
Wonderful,159,,the mystery of helium,Wonderful! 159: The Mystery of Helium \nPublis...,NaN,NaN
Move Your DNA,038,NaN,How Did You Move In 2015?,"[Episode 38, How Did You Move in 2015?, Your o...",SPEAKER: today’s show is about reflection and...,{'DANI': ['Today’s show is about reflection an...
Radiolab,NaN,2020,The Other Latif: Bonus Episode!,Announcer: Listener-supported WNYC Studios. ...,NaN,NaN


In [8]:
data.index.value_counts()

This American Life           732
Radiolab                     261
Welcome to Nightvale         170
Move Your DNA                108
The Allusionist               97
Bullseye with Jesse Thorn     63
MBMBaM                        32
Sawbones                      30
One Bad Mother                29
Wonderful                     29
Friendly Fire                 28
The Greatest Generation       28
Shmanners                     28
Judge John Hodgman            28
NeoScum                       20
The Adventure Zone            19
You're Wrong About            19
The Flophouse                 16
Switchblade Sisters           14
Unlocking Us                  12
Name: podcast, dtype: int64

In [9]:
data.loc[data.index=='NeoScum'].Text[0][:1000]

'Mike Migdall (MM): Hey everybody and welcome to the NeoScum actual play podcast!  Gannon Reedy (GR): Oh fuck yeah! [laughs]  [everyone laughs]   MM: You wanna start over?  [crosstalk; the theme to the podcast begins to play. It sounds sort of like the Full House theme a little bit? There’s a Great Saxophone Part in it.]  MM: Heeeeey!  GR: Heeeeeey! Aww, yes!   MM: Yes!!  GR: Welcome to NeoScum!  MM: A Shadowrun actual play roleplaying podcast.   GR: My name is Gannon Reedy.  MM: My name is Mike Migdall  GR: Dude… you’re listening to us, but do you even know what Shadowrun is?  MM: Do you? Well that’s why we’re here, to tell you what Shadowrun is.  GR: It’s like a futuristic D&D type game. I like to describe it as Blade Runner but with orcs.   MM: Woah. That’s cool, because I like to describe as Blade Runner except with orcs, elves, dwarves, and gnomes. And humans.  GR: Hmm. I guess that’s more accurate.  MM: And also centaur. I’m getting a “go longer” from my producer.  GR: Okay. Go l

In [10]:
# get rid of texts less than 6500 characters
podcast_df = pd.DataFrame()

for i in range(len(data)):
    if len(data.iloc[i, 3]) > 6500:  # I kept changing this number to see what returned, this gets rid of the erroneous text
        podcast_df = podcast_df.append(data.iloc[i, :])

In [11]:
podcast_df

,Anonymized_text,Episode,Speaker_parts,Text,Title,Year
Welcome to Nightvale,NaN,000a,NaN,"CECIL: As a matter of fact, the facts don’t ma...",matter of blood part 2,2018
Welcome to Nightvale,NaN,001,NaN,"And now the news. Old Woman Josie, out near th...",pilot,2012
Welcome to Nightvale,NaN,002,NaN,"And now, the news. Have any of our listeners s...",glow cloud,2012
Welcome to Nightvale,NaN,003,NaN,The Night Vale Business Association is proud t...,station management,2012
Welcome to Nightvale,NaN,004,NaN,The Pteranodons mostly attacked women with gla...,pta meeting,2012
...,...,...,...,...,...,...
The Allusionist,NaN,106,NaN,hear this episode and read more about it at ...,typo demon,2019
The Allusionist,NaN,107,NaN,visit theREDACTED.org/apples to hear this ep...,apples,2019
The Allusionist,NaN,108,NaN,vist theREDACTED.org/enjoy to hear this epis...,enjoy!,2019
The Allusionist,NaN,109,NaN,visit theREDACTED.org/eastwest to hear this ...,east west,2019


In [13]:
podcast_df.sample(10)

,Anonymized_text,Episode,Speaker_parts,Text,Title,Year
This American Life,NaN,482,NaN,"Prologue Ira Glass OK, everybody. ...","Lights, Camera, Christmas!",NaN
This American Life,NaN,330,NaN,Prologue Ira Glass So Robyn was the...,My Reputation,NaN
Welcome to Nightvale,NaN,019a,NaN,Blinking red light in the night sky. The futur...,the sandstorm,2013
This American Life,NaN,238,NaN,Prologue Ira Glass Alex spoke Russi...,Lost in Translation,NaN
This American Life,NaN,360,NaN,Prologue Ira Glass Back in 1994-- ...,Switched at Birth,NaN
This American Life,NaN,298,NaN,"Prologue Ira Glass Marti was 22, ju...",Getting and Spending,NaN
The Allusionist,NaN,028,NaN,"to read about or hear this episode, visit the...",wltm part i,2016
One Bad Mother,﻿ SPEAKER hi. i’m SPEAKER. SPEAKER and i...,386,"{'caller': ['hey, biz and theresa! i am callin...",biz Hi. I’m Biz. theresa And I’m Theresa. biz ...,The 5th Ever Holiday Genius Fail Spectacular ...,NaN
Friendly Fire,NaN,130,NaN,\nNote: This show periodically replaces thei...,final draft,
This American Life,NaN,665,NaN,Prologue: Prologue Ira Glass At on...,Before Things Went to Hell,NaN


### Podcast-specific info

Podcast-specific columns:  
    *- # of hosts - 0.5 indicates a whether or not the podcast regularly has guests
    *- genre, topic
    *- scripted/unscripted
    *- fiction/nonfiction
    *- format (interview, chat, etc)
        *- chat is generally talking about things, recap is talking about a specific thing (tv show, movie, etc)
    *- rating (itunes)
        *- I anticipated all ratings to be 4 or above, not 4.6 or above.  That teeny-tiny margin doesn't seem like it will be useful for analysis

This has been the hardest part of the project so far.  A lot of these categories are open to interpretation.

In [14]:
pod_feats = [['Welcome to Nightvale', 1, ['comedy', 'sci-fi'], 'scripted', 'fiction', 'news', 4.8],
             ['Move Your DNA', 1.5, ['health', 'fitness'], 'unscripted', 'nonfiction', 'chat', 4.8],
             ['You\'re Wrong About', 2, ['history', 'education'], 'unscripted', 'nonfiction', 'chat', 4.6],
             ['Unlocking Us', 1.5, ['health', 'lifestyle'], 'unscripted', 'nonfiction', 'interview', 4.6],
             ['Radiolab', 2, ['society', 'education'], 'unscripted', 'nonfiction', 'storytelling', 4.7],
             ['This American Life', 1.5, ['society','history'], 'unscripted', 'nonfiction', 'storytelling', 4.6],
             ['Bullseye with Jesse Thorn' , 1.5, ['comedy', 'society'], 'unscripted', 'nonfiction', 'interview', 4.7],
             ['One Bad Mother', 2.5, ['comedy', 'parenting'], 'unscripted', 'nonfiction', 'chat', 4.7],
             ['Judge John Hodgman', 1.5, ['comedy, advice'], 'unscripted', 'nonfiction', 'chat', 4.8],
             ['The Flophouse' , 3, ['comedy', 'movies'], 'unscripted', 'nonfiction', 'recap', 4.8],
             ['Switchblade Sisters', 1.5, ['comedy', 'movies'], 'unscripted', 'nonfiction', 'chat', 4.9],
             ['MBMBaM', 3, ['comedy','advice'], 'unscripted', 'nonfiction', 'chat', 4.9],
             ['Sawbones', 2, ['history', 'medicine'], 'unscripted', 'nonfiction', 'storytelling', 4.8],
             ['Wonderful', 2, ['comedy', 'society'], 'unscripted', 'nonfiction', 'chat', 4.9],
             ['The Greatest Generation', 2, ['comedy', 'TV'], 'unscripted', 'nonfiction', 'recap', 4.9],
             ['Friendly Fire', 3, ['history', 'movies'], 'unscripted', 'nonfiction', 'recap', 4.6],
             ['Shmanners', 2, ['society', 'advice'], 'unscripted', 'nonfiction', 'chat', 4.8],
             ['The Adventure Zone', 4, ['games', 'RP'], 'unscripted', 'fiction', 'LARP', 4.9],
             ['NeoScum', 5, ['games', 'RP'], 'unscripted', 'fiction', 'LARP', 4.9],
             ['The Allusionist', 1, ['education', 'language'], 'scripted', 'nonfiction','storytelling', 4.8]]

# In case you're a cool person reading this and don't know, LARP is live action role playing.

In [15]:
pod_feats_df = pd.DataFrame(pod_feats, columns = ['podcast', 'Hosts', 'Genre-Topic', 
                                                  'Scripted/Un', 'Fiction/Non', 
                                                  'Format', 'Rating']).set_index('podcast')
pod_feats_df

,Hosts,Genre-Topic,Scripted/Un,Fiction/Non,Format,Rating
podcast,,,,,,
Welcome to Nightvale,1.0,"[comedy, sci-fi]",scripted,fiction,news,4.8
Move Your DNA,1.5,"[health, fitness]",unscripted,nonfiction,chat,4.8
You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6
Unlocking Us,1.5,"[health, lifestyle]",unscripted,nonfiction,interview,4.6
Radiolab,2.0,"[society, education]",unscripted,nonfiction,storytelling,4.7
This American Life,1.5,"[society, history]",unscripted,nonfiction,storytelling,4.6
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7
One Bad Mother,2.5,"[comedy, parenting]",unscripted,nonfiction,chat,4.7
Judge John Hodgman,1.5,"[comedy, advice]",unscripted,nonfiction,chat,4.8


In [16]:
podcast_df = pod_feats_df.join(podcast_df, on='podcast', sort=True)
podcast_df.sample(10)

,Hosts,Genre-Topic,Scripted/Un,Fiction/Non,Format,Rating,Anonymized_text,Episode,Speaker_parts,Text,Title,Year
podcast,,,,,,,,,,,,
Friendly Fire,3.0,"[history, movies]",unscripted,nonfiction,recap,4.6,NaN,134,NaN,Note: This show periodically replaces their ad...,final draft,
This American Life,1.5,"[society, history]",unscripted,nonfiction,storytelling,4.6,NaN,470,NaN,"Prologue Ira Glass OK, so there's ...",Show Me the Way,NaN
The Adventure Zone,4.0,"[games, RP]",unscripted,fiction,LARP,4.9,NaN,167,NaN,"The Adventure Zone: Graduation – Ep. 24, With ...",graduation24 with frenemies like this,
Welcome to Nightvale,1.0,"[comedy, sci-fi]",scripted,fiction,news,4.8,NaN,029,NaN,This episode was co-written with Russel Swense...,subway,2013
The Allusionist,1.0,"[education, language]",scripted,nonfiction,storytelling,4.8,NaN,099,NaN,visit theREDACTED.org/polari to listen to th...,polari,2019
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,"﻿ SPEAKER gentle, trilling SPEAKER with a st...",,"{'like A Mighty Wind': [], 'catherine o’hara':...","music Gentle, trilling music with a steady dru...",Catherine O’Hara,NaN
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,"﻿ SPEAKER gentle, trilling SPEAKER with a st...",,"{'roger': ['angell thanks so much, jesse! gla...","music Gentle, trilling music with a steady dru...","Happy 100th birthday, Roger Angell",NaN
This American Life,1.5,"[society, history]",unscripted,nonfiction,storytelling,4.6,NaN,388,NaN,Act One: Act One Ira Glass Some ki...,Rest Stop,NaN
One Bad Mother,2.5,"[comedy, parenting]",unscripted,nonfiction,chat,4.7,﻿0 SPEAKER ellis hi. i’m SPEAKER. 0 SPEAKE...,378,{'caller': ['i… am doing okay. i am calling wi...,biz ellis Hi. I’m Biz. theresa thorn And I’m T...,"Sometimes Daddy Cries, with Guest Todd Rennebohm",NaN


In [17]:
podcast_df.index.value_counts()

This American Life           731
Radiolab                     192
Welcome to Nightvale         168
The Allusionist               97
Bullseye with Jesse Thorn     63
MBMBaM                        32
Sawbones                      30
Wonderful                     29
One Bad Mother                29
Friendly Fire                 28
Shmanners                     28
The Greatest Generation       28
Judge John Hodgman            28
NeoScum                       20
The Adventure Zone            19
You're Wrong About            19
The Flophouse                 16
Switchblade Sisters           14
Unlocking Us                  12
Move Your DNA                  1
Name: podcast, dtype: int64

### Text-processing functions

In [18]:
# anticipating a log of ugly floats
def percent(decimal):
    decimal *= 100
    percentage = '{:.3f}'.format(decimal)
    percentage = float(percentage)
    return percentage

percent(0.45615981981)

45.616

In [19]:
ord("'")

39

In [20]:
mini = podcast_df.sample(5)
tokens = mini.Text.map(nlp)
tokens

podcast
You're Wrong About         (Sarah, :,   , I, want, to, know, about, priso...
The Greatest Generation    (Note, :, This, show, periodically, replaces, ...
The Flophouse              (music, Light, ,, up, -, tempo, ,, electric, g...
This American Life         (   , Prologue, :, Prologue,        , Ira, Gla...
This American Life         (   , Prologue,        , Ira, Glass,  , From, ...
Name: Text, dtype: object

In [21]:
# the text is still really ugly - time to unnecessarily complicate text cleaning
# To get rid of a bunch of unnecessary spaces and undesired characters
characters = [(48, 58), (65, 91), (97, 123)]
asciis = []
for (b, e) in characters:
    for i in range(b, e):
        asciis.append(i)

wanted_characters = ['.','?','!',':',';','%','&','\'',',','"','-','[',']']
for c in wanted_characters:
    asciis.append(ord(c))
    
def textcleaner(text):
    asciis = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 72, 73, 
              74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 
              99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 
              114, 115, 116, 117, 118, 119, 120, 121, 122, 46, 63, 33, 58, 59, 37, 38, 39, 
              44, 34, 45, 91, 93]
    text = text.split(' ')
    new_text = []
    
    for t in text:
        if len(t) > 0:
            all_ascii = True
            for c in t:
                if ord(c) not in asciis:
                    all_ascii = False
                    continue
                continue
            if all_ascii == True: new_text.append(t)
        
    text = ' '.join(new_text)
    
    return text
    

print('Original:  ', mini.Text[1][:1000])
print('\n')
print('Cleaned: ', textcleaner(mini.Text[1][:1000]))


Original:   Note: This show periodically replaces their ad breaks with new promotional clips. Because of this, both the 
transcription for the clips and the timestamps after them may be inaccurate at the time of viewing this 
transcript. 
00:00:00  Music  Transition  Dark Materia’s “The Picard Song,” record-scratching into a Sisko-
centric remix by Adam Ragusea. 
 
Picard: Here’s to the finest crew in Starfleet! Engage. 
 
[Music begins. A fast-paced techno beat.] 
 
Picard: Captain Jean-Luc Picard, the USS Enterprise! 
 
[Music slows, record scratch, and then music speeds back up.] 
 
Sisko: Commander Benjamin Sisko, the Federation starbase... 
Deep Space 9. 
 
[Music ends.] 
00:00:14  Music  Music  Record scratch back into "The Picard Song," which plays quietly in 
the background. 
00:00:15  Ben Harrison  Host  Welcome to The Greatest Generation: Deep Space Nine. It's a Star 
Trek podcast by a couple of guys who are a little bit embarrassed to 
have a Star Trek podcast. [slightly mut

In [22]:
mini['Text'] = mini.Text.map(textcleaner)

In [23]:
mini.Text[1][:1000]

'Note: This show periodically replaces their ad breaks with new promotional clips. Because of this, both the for the clips and the timestamps after them may be inaccurate at the time of viewing this Music Transition Dark Picard record-scratching into a remix by Adam Ragusea. to the finest crew in Starfleet! Engage. begins. A fast-paced techno beat.] Captain Jean-Luc Picard, the USS Enterprise! slows, record scratch, and then music speeds back up.] Commander Benjamin Sisko, the Federation starbase... Space 9. ends.] Music Music Record scratch back into "The Picard Song," which plays quietly in background. Ben Harrison Host Welcome to The Greatest Generation: Deep Space Nine. It\'s a Star podcast by a couple of guys who are a little bit embarrassed to a Star Trek podcast. [slightly muttering] I\'m Ben Harrison. Adam Host I\'m Adam Pranica. Ben Host Tried to put a little embarrassment into my performance of my own "I\'m... [mumbling] Ben Harrison." Adam Host I could hear it! Ben Host Yeah

In [25]:
podcast_df

,Hosts,Genre-Topic,Scripted/Un,Fiction/Non,Format,Rating,Anonymized_text,Episode,Speaker_parts,Text,Title,Year
podcast,,,,,,,,,,,,
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,"﻿ SPEAKER hey all, it’s SPEAKER. as 2020 dra...",,"{'brian': ['heater adrian tomine, thank you s...","jesse thorn Hey all, it’s Jesse. As 2020 draws...",Cartoonist and Author Adrian Tomine,NaN
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,"﻿ SPEAKER gentle, trilling SPEAKER with a st...",,"{'cristin': ['milioti hi!', 'you know what? i...","music Gentle, trilling music with a steady dru...","Cristin Milioti on ‘Palm Springs,’ ‘How I Met...",NaN
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,"﻿ SPEAKER gentle, trilling SPEAKER with a st...",,{'frank': ['turner thank you for having me! v...,"music Gentle, trilling music with a steady dru...",Musician Frank Turner,NaN
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,"﻿ muSPEAKERc gentle, trilling muSPEAKERc wit...",,"{'s.': ['und effect muc elland fade', 'und ef...","music Gentle, trilling music with a steady dru...",Isiah Whitlock Jr.,NaN
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,"﻿ SPEAKER gentle, trilling SPEAKER with a st...",,{'anna': ['konkle i don’t know if it’s ever ...,"music Gentle, trilling music with a steady dru...","Maya Erskine and Anna Konkle of PEN15, back f...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,"HOST : Ooh, I have one, I have one I have one...",,{'Mike': ['Hm. Sarah : And so I’m not invest...,"Mike : Ooh, I have one, I have one I have one...","Bonus: ""The Dark Knight""",2021
You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,HOST: The point of shaming someone partly is...,,"{'Mike': ['The nice, Save the Best for Last ...",Sarah: The point of shaming someone partly i...,Vanessa Williams Part 2: Saving The Best For Last,2021
You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,HOST: Economically like it was once necessar...,,"{'Mike': ['Welcome to You're Wrong About , ...",Sarah: Economically like it was once necessa...,The O.J. Simpson Trial: From the Mixed-Up File...,2021


In [26]:
one_host_pods = ['You\'re Wrong About',
            'The Flophouse','MBMBaM','Sawbones','Wonderful','The Greatest Generation',
            'Friendly Fire','Shmanners','The Adventure Zone','Neoscum']

In [27]:
same_hosts = podcast_df[podcast_df.index.isin(one_host_pods)]

In [28]:
same_hosts.index.value_counts()

MBMBaM                     32
Sawbones                   30
Wonderful                  29
Shmanners                  28
Friendly Fire              28
The Greatest Generation    28
The Adventure Zone         19
You're Wrong About         19
The Flophouse              16
Name: podcast, dtype: int64

In [29]:
%store same_hosts

Stored 'same_hosts' (DataFrame)


## Analysis
Columns: Tokens, Top50, Token_count, Avg_token_len, TTR, kband, Bigrams, Bigram_top25, Sent_toks, Avg_sent_len, POS_frequency, POS_length, Verb_lemmas, Ent_count

In [26]:
# add Tokens column
podcast_df['Tokens'] = podcast_df.Text.map(nlp)

In [27]:
podcast_df.Tokens.loc['Welcome to Nightvale'][2][:500]

And now, the news. Have any of our listeners seen the glowing cloud that has been moving in from the west? Well, John Peters, you know, the farmer? He saw it over the Western Ridge this morning, said he would have thought it was the setting sun if it for the time of day. Apparently the cloud glows in a variety of colors, perhaps changing from observer to observer, although all report a low whistling when it draws near. One death has already been attributed to the glowcloud. But listen, probably nothing. If we had to shut down the town for every mysterious event that at least one death could be attributed to, never have time to do anything, right? what the Secret Police are saying, and I agree, although I would not go so far as to endorse their suggestion to directly at the cloud, shrieking and waving your arms, just to see what it The Apache Tracker, and I remind you that this is that white guy who wears the huge and cartoonishly inaccurate Indian headdress, has announced that he has f

In [28]:
def top50(Tokens):
    counts = Counter(t.text for t in Tokens if t.is_alpha)
    return counts.most_common(50)

In [29]:
# add Top50 column
podcast_df['Top50'] = podcast_df.Tokens.map(top50)

In [30]:
# add token count (transcript_length) column
podcast_df['Token_count'] = podcast_df.Tokens.map(len)

In [31]:
def word_len(Tokens):
    if len(Tokens) > 10:
        lengths = [(w, len(w.text)) for w in Tokens if w.is_alpha]
    else:
        lengths = [('null',0)]
    
    avg = statistics.mean([l[-1] for l in lengths])
    
    return lengths, avg

In [32]:
# add token length column
podcast_df['Token_lengths'] = podcast_df.Tokens.map(lambda x: word_len(x)[0])

In [33]:
# add average token length column
podcast_df['Avg_token_len'] = podcast_df.Tokens.map(lambda x: word_len(x)[1])

In [34]:
# TTR
def get_ttr(Tokens):
    if len(Tokens) > 1:
        lower = [t.text.lower() for t in Tokens if t.is_alpha]
        ttr = percent(len(set(lower))/len(lower))
    else:
        ttr = 0
        
    return ttr

In [35]:
# add TTR column
podcast_df['TTR'] = podcast_df.Tokens.map(get_ttr)

In [36]:
# read in k-bands
import pickle
f = open('data/goog_kband.pkl','rb')
goog_kband = pickle.load(f)
f.close()

goog_kband['throughout']

2

In [37]:
def get_kband(Tokens):
    if len(Tokens) > 1:
        kbands = []
        for t in Tokens:
            if t.lemma_ in goog_kband:
                kbands.append((t, goog_kband[t.lemma_]))
        avg_kband = statistics.mean([t[1] for t in kbands])
    else:
        kbands = 0
        avg_kband = 0
    
    return kbands, avg_kband

In [38]:
# add ('word', kband) column
podcast_df['kband'] = podcast_df.Tokens.map(lambda x: get_kband(x)[0])

In [39]:
# add average kband column
podcast_df['Avg_kband'] = podcast_df.Tokens.map(lambda x: get_kband(x)[1])

In [40]:
def bigrams(Tokens):
    if len(Tokens) > 1:
        bigrams = []
        for t in Tokens[:-1]:
            if t.text.isalpha() and Tokens[t.i + 1].text.isalpha():
                bigram = (t.text.lower(), Tokens[t.i + 1].text.lower())
                bigrams.append(bigram)
        counts = Counter(b for b in bigrams).most_common(25)
    else:
        bigrams = 'null'
        
    return bigrams

In [41]:
# add bigrams column
podcast_df['Bigrams'] = podcast_df.Tokens.map(lambda x: bigrams(x))

In [42]:
# add 25 most common bigram column
podcast_df['Bigram_top25'] = podcast_df.Bigrams.map(lambda x: Counter(x).most_common(25))

In [43]:
# add (token, part-of-speech) column
podcast_df['POS'] = podcast_df.Tokens.map(lambda t: [(w, w.pos_) for w in t])

In [44]:
# weighs pos frequency against total text length
def POS_frequency(POS_text):
    counts = Counter(elem[-1].upper() for elem in POS_text)
    total = len(POS_text)
    
    pos_freq = {}
    for (pos, count) in counts.items():
        pos_freq[pos] = percent(count/total)
        
    return pos_freq

In [45]:
# add {part-of-speech: frequency} column
podcast_df['POS_freq'] = podcast_df.POS.map(POS_frequency)

In [46]:

podcast_df.POS_freq[1]
spacy.explain('SCONJ')

{'NOUN': 11.396,
 'ADJ': 4.793,
 'PUNCT': 16.612,
 'VERB': 13.44,
 'ADP': 8.435,
 'DET': 8.388,
 'PROPN': 3.842,
 'CCONJ': 3.689,
 'AUX': 4.147,
 'X': 0.482,
 'SCONJ': 1.234,
 'ADV': 7.331,
 'INTJ': 3.125,
 'PRON': 10.726,
 'PART': 1.656,
 'NUM': 0.705}

'subordinating conjunction'

In [47]:
# add part-of-speech frequency columns
podcast_df['Noun_freq'] = podcast_df.POS_freq.map(lambda x: x.get('NOUN', 'null'))
podcast_df['Proper_noun_freq'] = podcast_df.POS_freq.map(lambda x: x.get('PROPN', 'null'))
podcast_df['Verb_freq'] = podcast_df.POS_freq.map(lambda x: x.get('VERB', 'null'))
podcast_df['Adj_freq'] = podcast_df.POS_freq.map(lambda x: x.get('ADJ', 'null'))
podcast_df['Adv_freq'] = podcast_df.POS_freq.map(lambda x: x.get('ADV', 'null'))
podcast_df['Interjection_freq'] = podcast_df.POS_freq.map(lambda x: x.get('INTJ', 'null'))
podcast_df['Preposition_freq'] = podcast_df.POS_freq.map(lambda x: x.get('ADP', 'null'))
podcast_df['Conjunction_freq'] = podcast_df.POS_freq.map(lambda x: x.get('SCONJ', 'null'))

In [48]:
podcast_df.sample(5)

,Hosts,Genre-Topic,Scripted/Un,Fiction/Non,Format,Rating,Episode,Host_speech,Text,Title,...,POS,POS_freq,Noun_freq,Proper_noun_freq,Verb_freq,Adj_freq,Adv_freq,Interjection_freq,Preposition_freq,Conjunction_freq
podcast,,,,,,,,,,,,,,,,,,,,,
Radiolab,2.0,"[society, education]",unscripted,nonfiction,storytelling,4.7,NaN,NaN,LN: Latif Nasser RK: Robert Krulwich JA: Jad A...,Oliver Sipple,...,"[(LN, NOUN), (:, PUNCT), (Latif, PROPN), (Nass...","{'NOUN': 12.853, 'PUNCT': 17.793, 'PROPN': 7.7...",12.853,7.710,10.885,3.555,5.649,1.790,7.558,1.360
This American Life,1.5,"[society, history]",unscripted,nonfiction,storytelling,4.6,568,NaN,"Prologue Ira Glass In October 2003, a guy was ...",Human Spectacle 2015,...,"[(Prologue, PROPN), (Ira, PROPN), (Glass, PROP...","{'PROPN': 10.401, 'ADP': 8.062, 'NUM': 1.052, ...",12.651,10.401,11.566,4.096,4.953,1.174,8.062,1.230
This American Life,1.5,"[society, history]",unscripted,nonfiction,storytelling,4.6,303,NaN,Prologue Ira Glass It's a classic David and Go...,David and Goliath,...,"[(Prologue, PROPN), (Ira, PROPN), (Glass, PROP...","{'PROPN': 8.915, 'PRON': 9.818, 'AUX': 6.115, ...",13.686,8.915,12.010,4.448,5.489,0.672,7.957,1.262
This American Life,1.5,"[society, history]",unscripted,nonfiction,storytelling,4.6,437,NaN,Prologue Ira Glass When I sat down to intervie...,Old Boys Network,...,"[(Prologue, PROPN), (Ira, PROPN), (Glass, PROP...","{'PROPN': 9.173, 'ADV': 5.315, 'PRON': 10.864,...",13.380,9.173,12.530,4.049,5.315,0.475,8.173,1.483
The Allusionist,1.0,"[education, language]",scripted,nonfiction,storytelling,4.8,012,NaN,"To hear this episode or read more about it, vi...",Pride,...,"[(To, PART), (hear, VERB), (this, DET), (episo...","{'PART': 3.005, 'VERB': 11.478, 'DET': 9.557, ...",16.798,5.911,11.478,6.502,5.517,0.197,10.542,0.837


In [49]:
def POS_length(POS_text):
    pos_dict = {'NOUN': 0, 'VERB': 0, 'ADV': 0, 'ADJ': 0}
    pron_dict = {'i': 0, 'you': 0, 'she': 0, 'he': 0, 'it': 0, 'they': 0, 'we': 0}
    for (token, pos) in POS_text:
        if pos in pos_dict.keys():
            pos_dict[pos] = (pos_dict[pos] + len(token.text))/2
        if token.text in pron_dict.keys():
            pron_dict[token.text] = pron_dict[token.text] + 1
    
    if sum(pron_dict.values()) != 0:
        pron_total = sum(pron_dict.values())
    
    if sum(pron_dict.values()) != 0:
        for (p, c) in pron_dict.items():
            pron_dict[p] = percent(c/pron_total)
    
    
    return pos_dict, pron_dict

# Average word length of each POS
# POS_length[0][0] = noun
#           [0][1] = verb
#           [0][2] = adv
#           [0][3] = adj

# Individual pronoun occurrence weighed against total # of pronouns
# POS_length[1][1] = 'i'
#           [1][2] = 'you'
#           [1][3] ='she'
#           [1][4] = 'he'
#           [1][5] = 'it'
#           [1][6] = 'they'
#           [1][7] = 'we'


In [50]:
podcast_df['POS_length'] = podcast_df.POS.map(lambda p: POS_length(p)[0])

In [51]:
podcast_df['Avg_noun_len'] = podcast_df.POS_length.map(lambda d: d['NOUN'])
podcast_df['Avg_verb_len'] = podcast_df.POS_length.map(lambda d: d['VERB'])
podcast_df['Avg_adj_len'] = podcast_df.POS_length.map(lambda d: d['ADJ'])
podcast_df['Avg_adv_len'] = podcast_df.POS_length.map(lambda d: d['ADV'])

In [52]:
podcast_df['Pron_counts'] = podcast_df.POS.map(lambda p: POS_length(p)[1])

In [53]:
podcast_df['i_count'] = podcast_df.Pron_counts.map(lambda d: d['i'])
podcast_df['you_count'] = podcast_df.Pron_counts.map(lambda d: d['you'])
podcast_df['she_count'] = podcast_df.Pron_counts.map(lambda d: d['she'])
podcast_df['he_count'] = podcast_df.Pron_counts.map(lambda d: d['he'])
podcast_df['it_count'] = podcast_df.Pron_counts.map(lambda d: d['it'])
podcast_df['they_count'] = podcast_df.Pron_counts.map(lambda d: d['they'])
podcast_df['we_count'] = podcast_df.Pron_counts.map(lambda d: d['we'])

In [54]:
podcast_df.POS_freq[0]
podcast_df.Noun_freq[0]

{'X': 0.341,
 'NOUN': 13.732,
 'INTJ': 1.169,
 'DET': 9.674,
 'PUNCT': 13.423,
 'PROPN': 3.717,
 'ADP': 10.047,
 'NUM': 0.552,
 'VERB': 13.439,
 'PRON': 10.047,
 'ADJ': 5.421,
 'PART': 1.461,
 'AUX': 3.457,
 'SCONJ': 1.737,
 'CCONJ': 4.204,
 'ADV': 7.58}

13.732

In [55]:
# weighs occurrence of each ent against total text length
def ent_counter(Tokens):
    counts = Counter(elem.label_ for elem in Tokens.ents)
    # print(sum(counts.values()))
    
    ent_counter = {}
    for (ent, value) in counts.items():
        ent_counter[ent] = percent(value/len(Tokens))
    # print(sum(ent_counter.values()))
    
    return ent_counter

In [56]:
podcast_df.groupby('podcast').Token_count.min()

podcast
Bullseye with Jesse Thorn     1552
Friendly Fire                10843
Judge John Hodgman            9300
MBMBaM                       11778
Move Your DNA                 2018
NeoScum                      11944
One Bad Mother               10217
Radiolab                      2035
Sawbones                      7123
Shmanners                     6931
Switchblade Sisters           7893
The Adventure Zone            7728
The Allusionist               1511
The Flophouse                15266
The Greatest Generation      13213
This American Life            8152
Unlocking Us                  4983
Welcome to Nightvale          2074
Wonderful                     7505
You're Wrong About            5542
Name: Token_count, dtype: int64

In [57]:
# how do I read all the text??
podcast_df.loc[podcast_df.Token_count==1951].Text[:]

Series([], Name: Text, dtype: object)

In [58]:
sum(podcast_df.Token_count)

17024395

In [59]:
# most common verb lemmas
def verb_lemmas(POS_text):
    counts = Counter(elem[0].lemma_ for elem in POS_text if elem[1] == 'VERB')
    
    verb_counter = {}
    for (verb, value) in counts.most_common(20):
        verb_counter[verb] = percent(value/sum(counts.values()))
        
    return verb_counter

verb_lemmas(podcast_df.POS[1])
# spacy thinks that an apostrophe is a verb?  Wonder why.

{'be': 9.965,
 'know': 9.266,
 'do': 5.682,
 'think': 5.07,
 'have': 3.409,
 'feel': 3.234,
 'mean': 2.535,
 'cristin': 2.535,
 'get': 2.448,
 'see': 2.36,
 'say': 1.923,
 'gon': 1.923,
 'laugh': 1.836,
 'go': 1.573,
 'watch': 1.573,
 'love': 1.573,
 'start': 1.399,
 'agree': 1.311,
 'want': 1.224,
 'talk': 1.136}

In [60]:
# add verb_lemmas column
podcast_df['verb_lemmas'] = podcast_df.POS.map(verb_lemmas)

In [61]:
# add sent_toks column
podcast_df['Sent_toks'] = podcast_df.Text.map(nltk.sent_tokenize)

In [62]:
# minor alteration to unit_len
def sent_len(doc):
    sentlens = []
    for c in doc:
        length = len([l for l in c.split()])
        sentlens.append((c, length))
        
    return sentlens

In [63]:
sent_len(podcast_df.Sent_toks[0][:10])

[('jesse thorn Hey all, Jesse.', 5),
 ('As 2020 draws to a close, think about what thankful for, other willing to drawing to a close.',
  18),
 ('What got you through the year?', 6),
 ('Odds are, if hearing my voice, public radio was one of the things.', 13),
 ('Public radio gave you accurate, dependable news about the election on the pandemic, information about local stories that matter to you.',
  21),
 ('You got fun and fascinating interviews from shows like Bullseye .', 11),
 ('If you wanna show your gratitude at the end of this year, consider supporting your local public radio station.',
  19),
 ('Public radio stations really need your help right now, more than ever.',
  12),
 ('And really easy to do!', 5),
 ('Just go to and give whatever you can.', 8)]

In [64]:
podcast_df['Sent_length'] = podcast_df.Sent_toks.map(sent_len)

How to extract host names??  Some are full names and some are just first names.  Maybe compare top 50 tokens and enable/kbands.  Any word thats in top50 and not kband 1 is likely a name (right?  maybe?)

Even after the helpful tips from Sonia and Na-Rae, I can't pickle this dataframe.  I *was* able to import a simple test datframe on crc, but apparently dataframes with spacy objects can't be pickled.  I get the following error message when I attempt to pickle it:  "NotImplementedError: \[E111\] Pickling a token is not supported, because tokens are only views of the parent Doc and can't exist on their own. A pickled token would always have to include its Doc and Vocab, which has practically no advantage over pickling the parent Doc directly. So instead of pickling the token, pickle the Doc it belongs to."

I'll finish adding data points and columns, then convert everything to string and send it to crc when it's time to train the model. 

In [65]:
# text.ent doesn't seem to be very useful for finding host names, I'll try enable and most frequent
import pickle
f = open('data/goog_kband.pkl','rb')
goog_kband = pickle.load(f)
f.close()

goog_kband['throughout']

2

In [66]:
# add entity frequency column
def ent_counter(doc):
    ents = []    
    people = []
    length = len(doc)
    
    for ent in doc.ents:
        ents.append((ent.text, ent.label_))
        if ent.label_ == 'PERSON':
            people.append(ent.text.lower().strip())
    count = dict(Counter([ent[1] for ent in ents]))
    for label, c in count.items():
        count[label] = percent(c/length)
        
    return count, people

ent_counter(podcast_df.Tokens[0])[0]
        
        

{'PERSON': 0.828,
 'DATE': 0.341,
 'ORG': 0.665,
 'WORK_OF_ART': 0.13,
 'CARDINAL': 0.227,
 'ORDINAL': 0.049,
 'PERCENT': 0.049,
 'NORP': 0.016,
 'GPE': 0.081,
 'TIME': 0.097,
 'PRODUCT': 0.016}

In [67]:
podcast_df['Ents'] = podcast_df.Tokens.map(lambda x: (ent_counter(x)[0], ent_counter(x)[-1]))
podcast_df.Ents[2][0]

{'PERSON': 1.313,
 'ORG': 0.424,
 'WORK_OF_ART': 0.068,
 'GPE': 0.273,
 'NORP': 0.191,
 'DATE': 0.273,
 'CARDINAL': 0.328,
 'LOC': 0.068,
 'ORDINAL': 0.205,
 'TIME': 0.055,
 'MONEY': 0.014,
 'QUANTITY': 0.014,
 'EVENT': 0.014,
 'PRODUCT': 0.014}

In [68]:
# VALUES ARE % OF ENTIRE PODCAST LENGTH
podcast_df['Organization'] = podcast_df.Ents.map(lambda x: x[0].get('ORG', 0))
podcast_df['Art'] = podcast_df.Ents.map(lambda x: x[0].get('WORK_OF_ART', 0))
podcast_df['Date'] = podcast_df.Ents.map(lambda x: x[0].get('DATE', 0))
podcast_df['Geopolitical'] = podcast_df.Ents.map(lambda x: x[0].get('GPE', 0))
podcast_df['Numbers'] = podcast_df.Ents.map(lambda x: x[0].get('CARDINAL', 0))
podcast_df['Event'] = podcast_df.Ents.map(lambda x: x[0].get('EVENT', 0))
podcast_df['Cash'] = podcast_df.Ents.map(lambda x: x[0].get('MONEY', 0))
podcast_df['Time'] = podcast_df.Ents.map(lambda x: x[0].get('TIME', 0))
podcast_df['Product'] = podcast_df.Ents.map(lambda x: x[0].get('PRODUCT', 0))

I originally wanted to try and do something with parsing out host names then singling out their speech.  This turned out to be a bit of a pipe dream, since A. spacy isn't very good at recognizing names in the somewhat chaotic transcripts and B. there's no way of differentiating between a speaker's tag (i.e. Jad Abumrad: talk talk talk) and a name just being mentioned in speech.

In [69]:
# my beautiful and *definitely* state-of-the-art spacy matcher pattern-maker.  I found it really annoying to have to
#      format a pattern matcher every time I wanted to look for something new
from spacy.matcher import Matcher

def pattern_maker():
    
    patterns = []
    add_items = True
    add_dict = True    
    
    while add_dict:        
        match_single = {}                    
            
        while add_items:       
            tag = input('enter tag (lowercase):  ').upper()
            if tag == '0':
                add_dict = False
                break

            string = input('enter string (all lowercase):  ')
            if string == 'true' or string == 'false':
                string = bool(string)

            if tag == 'POS':
                string = string.upper()
            
            match_single[tag] = string
            
            add_items = input('add more to this dict?(y/n)  ')
            if add_items == 'n':
                patterns.append(match_single)
                break
        
        continue
    
    if len(patterns) == 0:
        return
   
    return patterns

pattern_maker()

enter tag (lowercase):  0


In [70]:
def pattern_matcher(pattern, doc): 
    matcher = Matcher(nlp.vocab)
    matcher.add('pattern', [pattern])
    matches = matcher(doc)
    
    
    match_strings = []
    for match_id, start, end in matches:
        matched_span = doc[start:end].text
        match_strings.append(matched_span)
    # print('{} matches found'.format(len(match_strings)))
    return match_strings

pattern_matcher([{'IS_PUNCT': True}], podcast_df.Tokens[0])
# sorry about the giant flash of punctuation, I'm not sure why it's even printing

[',',
 '.',
 ',',
 ',',
 '.',
 '?',
 ',',
 ',',
 '.',
 ',',
 ',',
 '.',
 '.',
 ',',
 '.',
 ',',
 '.',
 '!',
 '.',
 '.',
 ',',
 '.',
 ':',
 '.',
 '[',
 '.',
 ']',
 ',',
 ',',
 '!',
 '.',
 '.',
 '.',
 ',',
 '.',
 ',',
 '.',
 ',',
 ':',
 ',',
 ',',
 '.',
 ',',
 '.',
 ',',
 ',',
 '.',
 ',',
 ',',
 '.',
 '.',
 '-',
 '.',
 ',',
 '.',
 ',',
 '.',
 ',',
 '.',
 '.',
 ',',
 ',',
 '.',
 '.',
 '.',
 '-',
 ',',
 ',',
 '-',
 '.',
 '[',
 '.',
 ']',
 '.',
 '.',
 '.',
 ',',
 '.',
 ',',
 '.',
 '.',
 '?',
 ',',
 '.',
 '?',
 '[',
 '.',
 ']',
 ',',
 '[',
 ']',
 ',',
 ',',
 '.',
 '.',
 ',',
 ',',
 ',',
 '-',
 '.',
 ',',
 '-',
 '-',
 ',',
 ',',
 ',',
 ',',
 '[',
 ']',
 ',',
 '.',
 '-',
 '.',
 ',',
 '-',
 ',',
 ',',
 '.',
 '?',
 '.',
 '%',
 '.',
 ',',
 ',',
 ',',
 ',',
 ',',
 '-',
 ',',
 '[',
 ']',
 '.',
 ',',
 '.',
 ',',
 '-',
 '-',
 '.',
 '[',
 ']',
 '.',
 ',',
 ',',
 '?',
 '[',
 ']',
 '.',
 ',',
 ',',
 '.',
 '.',
 ',',
 '.',
 ',',
 ',',
 ',',
 '.',
 ',',
 '.',
 ',',
 '[',
 ']',
 ',',
 ',',
 '.',
 '[',
 '.'

In [71]:
podcast_df['Punctuation'] = podcast_df.Tokens.map(lambda t: Counter([p for p in pattern_matcher([{'IS_PUNCT': True}], t) if p != ',']))
podcast_df.Punctuation[10]

Counter({'.': 368,
         ':': 18,
         '[': 46,
         ']': 46,
         '!': 29,
         '-': 7,
         '?': 26,
         ';': 1})

In [72]:
podcast_df['period_freq'] = podcast_df.Punctuation.map(lambda d: percent(d['.']/sum(d.values())))
podcast_df['excl_freq'] = podcast_df.Punctuation.map(lambda d: percent(d['!']/sum(d.values())))
podcast_df['quest_freq'] = podcast_df.Punctuation.map(lambda d: percent(d['?']/sum(d.values())))
podcast_df['hyph_freq'] = podcast_df.Punctuation.map(lambda d: percent(d['-']/sum(d.values())))
podcast_df.loc['MBMBaM', 'period_freq':'hyph_freq'][:20]
# numbers are percentage of all punctuation except commas

,period_freq,excl_freq,quest_freq,hyph_freq
podcast,,,,
MBMBaM,42.247,6.820,7.311,0.638
MBMBaM,38.707,9.938,7.466,0.999
MBMBaM,41.122,7.682,6.946,0.920
MBMBaM,40.131,7.387,6.079,1.539
MBMBaM,40.433,7.612,6.198,1.664
MBMBaM,37.143,7.930,6.006,0.583
MBMBaM,32.920,3.382,5.479,2.911
MBMBaM,35.348,3.480,4.853,4.075
MBMBaM,36.112,4.147,5.069,3.016


In [73]:
# make a regex for tags
expression = r'(?<=\[).+?(?=\])'
len(re.findall(expression, podcast_df.Text[100]))
re.findall(expression, podcast_df.loc['Welcome to Nightvale', 'Text'][0])[:20]

140

['phone rings', 'weather: in the by ']

In [74]:
# maybe get len and parts of speech within tags
podcast_df['Tags'] = podcast_df.Text.map(lambda text: re.findall(expression, text))
len(podcast_df.Tags[0])
podcast_df.Tags[0]

38

['Music fades out.',
 'Music fades in.',
 'Laughs.',
 'laughs',
 'chuckles',
 'laughs',
 'laughs',
 'laughs',
 'chuckles',
 'Adrian confirms.',
 'laughs',
 'chuckles',
 'Laughs.',
 'laughing',
 'laughs',
 'laughs',
 'Laughs.',
 'laughs',
 'laughs',
 'Sound of office telephone plays quietly in background.',
 'Music fades out.',
 'Laughs.',
 'Chuckling.',
 'laughs',
 'laughing',
 'chuckling',
 'laughs',
 'Adrian agrees with a laugh.',
 'laugh',
 'censored',
 'laugh',
 'Adrian laughs and agrees.',
 'Laughs.',
 'Laughs.',
 'laughs',
 'Adrian laughs.',
 'Beat.',
 'Music fades out.']

In [75]:
[t.split(' ') for t in podcast_df.Tags[0]]

[['Music', 'fades', 'out.'],
 ['Music', 'fades', 'in.'],
 ['Laughs.'],
 ['laughs'],
 ['chuckles'],
 ['laughs'],
 ['laughs'],
 ['laughs'],
 ['chuckles'],
 ['Adrian', 'confirms.'],
 ['laughs'],
 ['chuckles'],
 ['Laughs.'],
 ['laughing'],
 ['laughs'],
 ['laughs'],
 ['Laughs.'],
 ['laughs'],
 ['laughs'],
 ['Sound',
  'of',
  'office',
  'telephone',
  'plays',
  'quietly',
  'in',
  'background.'],
 ['Music', 'fades', 'out.'],
 ['Laughs.'],
 ['Chuckling.'],
 ['laughs'],
 ['laughing'],
 ['chuckling'],
 ['laughs'],
 ['Adrian', 'agrees', 'with', 'a', 'laugh.'],
 ['laugh'],
 ['censored'],
 ['laugh'],
 ['Adrian', 'laughs', 'and', 'agrees.'],
 ['Laughs.'],
 ['Laughs.'],
 ['laughs'],
 ['Adrian', 'laughs.'],
 ['Beat.'],
 ['Music', 'fades', 'out.']]

In [76]:
from spacy.tokenizer import Tokenizer

def avg_tag_length(tag_list):
    lengths = []
       
    for t in tag_list:
        t = t.split(' ')
        lengths.append(len(t))
       
    if len(lengths) > 0:
        avg_length = statistics.mean(lengths)
    else:
        avg_length = 0
    
    
    return avg_length
        
avg_tag_length(podcast_df.Tags[0])
    

1.631578947368421

In [77]:
podcast_df['Tag_len'] = podcast_df.Tags.map(avg_tag_length)
podcast_df.Tag_len[:20]

podcast
Bullseye with Jesse Thorn    1.631579
Bullseye with Jesse Thorn    3.561644
Bullseye with Jesse Thorn    4.361111
Bullseye with Jesse Thorn    2.133333
Bullseye with Jesse Thorn    2.511628
Bullseye with Jesse Thorn    4.205882
Bullseye with Jesse Thorn    1.714286
Bullseye with Jesse Thorn    1.972973
Bullseye with Jesse Thorn    1.967742
Bullseye with Jesse Thorn    4.185185
Bullseye with Jesse Thorn    2.195652
Bullseye with Jesse Thorn    2.500000
Bullseye with Jesse Thorn    2.044248
Bullseye with Jesse Thorn    3.135135
Bullseye with Jesse Thorn    3.263158
Bullseye with Jesse Thorn    2.144928
Bullseye with Jesse Thorn    1.945946
Bullseye with Jesse Thorn    1.923077
Bullseye with Jesse Thorn    2.722222
Bullseye with Jesse Thorn    2.725000
Name: Tag_len, dtype: float64

In [78]:
def tag_top_verb(tag_text_list):
    lemmas = []
    
    for t in tag_text_list:
        t = nlp(t)
        for token in t:
            if token.pos_=='VERB':
                lemmas.append(token.lemma_)    
    
    if len(lemmas) > 0:
        top_verb = Counter(lemmas).most_common(1)[0][0]
    else:
        top_verb = 'NaN'
        
    return top_verb
        
        
tag_top_verb(podcast_df.Tags[0])
    

'laugh'

In [79]:
podcast_df['Tag_top_verb'] = podcast_df.Tags.map(tag_top_verb)
podcast_df.Tag_top_verb[115]
# including this column definitely reflects confirmation bias, since I think that 
#      having "laugh" in a tag will weight it in favor of being comedy

'laugh'

In [80]:
# I apologize in advance to sensitive eyes and my parents
swears = ['fuck','shit','ass','asshole','damn','goddamnn','bitch','cunt']
for i in range(10):
    len([t.text for t in podcast_df.Tokens[i] if t.text in swears])

0

0

0

0

0

1

0

0

1

0

In [81]:
podcast_df['Swear_count'] = podcast_df.Tokens.map(lambda tokens: percent(len([t.text for t in tokens if t.text in swears])/len(tokens)))
podcast_df.Swear_count[-20:-10]

podcast
Wonderful             0.095
Wonderful             0.085
Wonderful             0.052
Wonderful             0.066
Wonderful             0.194
Wonderful             0.079
Wonderful             0.134
You're Wrong About    0.000
You're Wrong About    0.030
You're Wrong About    0.033
Name: Swear_count, dtype: float64

In [82]:
fake_swears = ['fudge','shoot','butthead','darn']
podcast_df['Fake_swear_count'] = podcast_df.Tokens.map(lambda tokens: percent(len([t.text for t in tokens if t.text in fake_swears])/len(tokens)))
podcast_df.Fake_swear_count[-20:-10]

podcast
Wonderful             0.024
Wonderful             0.009
Wonderful             0.021
Wonderful             0.000
Wonderful             0.000
Wonderful             0.000
Wonderful             0.034
You're Wrong About    0.000
You're Wrong About    0.010
You're Wrong About    0.008
Name: Fake_swear_count, dtype: float64

In [83]:
string = 'I thought that there are three more of I than you.  I felt good about this.  Do you feel okay?'
pattern_matcher([{'POS': 'PRON'}, {'POS': 'AUX', 'OP': '*'}, {'LEMMA': 'feel'}], nlp(string))
pattern_matcher([{'POS': 'PRON'}, {'POS': 'AUX', 'OP': '*'}, {'LEMMA': 'think'}], nlp(string))

['I felt', 'you feel']

['I thought']

In [84]:
def opinions(Tokens):
    count = 0
    
    count += len(pattern_matcher([{'POS': 'PRON'}, {'POS': 'AUX', 'OP': '*'}, {'LEMMA': 'feel'}], Tokens))
    count += len(pattern_matcher([{'POS': 'PRON'}, {'POS': 'AUX', 'OP': '*'}, {'LEMMA': 'think'}], nlp(string)))
    
    count /= len(Tokens)
    return count

opinions(nlp(string))
    

0.12

In [85]:
podcast_df['Opinion_count'] = podcast_df.Tokens.map(opinions)
podcast_df.Opinion_count[449]

0.0008193363375665711

In [86]:
def prep_per_sent(POSes, sent_toks):
    POSes = len([x[1] for x in POSes if x[1] == 'ADP'])
    adps = POSes/len(sent_toks)
    
    return adps

In [87]:
podcast_df['Prep_per_sent'] = podcast_df.apply(lambda x: prep_per_sent(POSes = x['POS'], sent_toks = x['Sent_toks']), axis=1)
podcast_df.Prep_per_sent[:157]

podcast
Bullseye with Jesse Thorn    1.733894
Bullseye with Jesse Thorn    1.132492
Bullseye with Jesse Thorn    1.839713
Bullseye with Jesse Thorn    1.229008
Bullseye with Jesse Thorn    0.890719
                               ...   
Move Your DNA                0.619485
Move Your DNA                1.231504
Move Your DNA                1.349544
Move Your DNA                1.267170
Move Your DNA                1.589165
Name: Prep_per_sent, Length: 157, dtype: float64

In [88]:
podcast_df['Donation_appeal'] = podcast_df.Tokens.map(lambda t: len(pattern_matcher([{'LEMMA':'donate', 'DEP': 'ROOT'}], t)))
podcast_df.Donation_appeal.value_counts()
# this doesn't look like it'll be particularly useful, but I made it so it can't hurt to leave it in

0    1605
1      64
2      10
3       2
Name: Donation_appeal, dtype: int64

In [89]:
sm = ['twitter','facebook','instagram','linkedin','twitch','tik tok']

def social_count(tokens):
    count = 0
    for t in tokens:
        if t.text in sm:
            count += 1
    
    return count

In [90]:
podcast_df['Social'] = podcast_df.Tokens.map(social_count)
podcast_df.Social.value_counts()
# same as last column: doesn't seem like it'd be particularly useful, but might be

0    1630
1      40
2       9
5       1
4       1
Name: Social, dtype: int64

In [91]:
podcast_df.columns

Index(['Hosts', 'Genre-Topic', 'Scripted/Un', 'Fiction/Non', 'Format',
       'Rating', 'Episode', 'Host_speech', 'Text', 'Title', 'Year', 'Tokens',
       'Top50', 'Token_count', 'Token_lengths', 'Avg_token_len', 'TTR',
       'kband', 'Avg_kband', 'Bigrams', 'Bigram_top25', 'POS', 'POS_freq',
       'Noun_freq', 'Proper_noun_freq', 'Verb_freq', 'Adj_freq', 'Adv_freq',
       'Interjection_freq', 'Preposition_freq', 'Conjunction_freq',
       'POS_length', 'Avg_noun_len', 'Avg_verb_len', 'Avg_adj_len',
       'Avg_adv_len', 'Pron_counts', 'i_count', 'you_count', 'she_count',
       'he_count', 'it_count', 'they_count', 'we_count', 'verb_lemmas',
       'Sent_toks', 'Sent_length', 'Ents', 'Organization', 'Art', 'Date',
       'Geopolitical', 'Numbers', 'Event', 'Cash', 'Time', 'Product',
       'Punctuation', 'period_freq', 'excl_freq', 'quest_freq', 'hyph_freq',
       'Tags', 'Tag_len', 'Tag_top_verb', 'Swear_count', 'Fake_swear_count',
       'Opinion_count', 'Prep_per_sent', 'Donat

In [92]:
for i in range(len(podcast_df.iloc[0, :])):
    print(podcast_df.columns[i])
    print(podcast_df.iloc[0, i])

Hosts
1.5
Genre-Topic
['comedy', 'society']
Scripted/Un
unscripted
Fiction/Non
nonfiction
Format
interview
Rating
4.7
Episode

Host_speech
nan
Text
jesse thorn Hey all, Jesse. As 2020 draws to a close, think about what thankful for, other willing to drawing to a close. What got you through the year? Odds are, if hearing my voice, public radio was one of the things. Public radio gave you accurate, dependable news about the election on the pandemic, information about local stories that matter to you. You got fun and fascinating interviews from shows like Bullseye . If you wanna show your gratitude at the end of this year, consider supporting your local public radio station. Public radio stations really need your help right now, more than ever. And really easy to do! Just go to and give whatever you can. And thanks. music Gentle, trilling music with a steady drumbeat plays under the dialogue. promo Speaker : Bullseye with Jesse Thorn is a production of MaximumFun.org and is distributed by

In [93]:
podcast_df['Top50'] = podcast_df.Top50.map(lambda x: set([d[0].lower() for d in x]))
podcast_df.Top50[10]

{'a',
 'about',
 'an',
 'and',
 'at',
 'carl',
 'did',
 'do',
 'for',
 'had',
 'have',
 'he',
 'her',
 'him',
 'his',
 'i',
 'in',
 'is',
 'it',
 'jesse',
 'know',
 'like',
 'me',
 'my',
 'of',
 'on',
 'one',
 'out',
 'reiner',
 'said',
 'she',
 'show',
 'so',
 'that',
 'the',
 'then',
 'there',
 'to',
 'was',
 'went',
 'what',
 'when',
 'with',
 'you'}

In [94]:
podcast_df['Know'] = podcast_df.verb_lemmas.map(lambda x: x.get('know', 0))
podcast_df['Be'] = podcast_df.verb_lemmas.map(lambda x: x.get('be', 0))
podcast_df['Do'] = podcast_df.verb_lemmas.map(lambda x: x.get('do', 0))
podcast_df['Mean'] = podcast_df.verb_lemmas.map(lambda x: x.get('mean', 0))
podcast_df['Make'] = podcast_df.verb_lemmas.map(lambda x: x.get('make', 0))
podcast_df['Go'] = podcast_df.verb_lemmas.map(lambda x: x.get('go', 0))

In [95]:
def avg_sent_len(sent_lens):
    counts = []
    
    for sent, length in sent_lens:
        counts.append(length)
    
    avg = statistics.mean(counts)
    
    return avg

avg_sent_len(podcast_df.Sent_length[0])

14.932773109243698

In [96]:
podcast_df['Avg_sent_len'] = podcast_df.Sent_length.map(avg_sent_len)

In [97]:
num_df = podcast_df[['Hosts','Rating','Token_count','Avg_token_len', 'Avg_sent_len', 'TTR','Avg_kband',
                     'Noun_freq','Proper_noun_freq','Verb_freq','Adj_freq','Adv_freq',
                     'Interjection_freq', 'Preposition_freq', 'Conjunction_freq', 'Avg_noun_len',
                     'Avg_verb_len', 'Avg_adj_len', 'Avg_adv_len', 'i_count', 'you_count', 'she_count',
                     'he_count', 'it_count', 'they_count', 'we_count', 'Know','Be', 'Do', 'Mean',
                     'Make', 'Go', 'Organization', 'Art', 'Date', 'Geopolitical', 'Numbers', 'Event',
                     'Cash', 'Time', 'Product', 'period_freq', 'excl_freq', 'quest_freq', 'hyph_freq',
                     'Tag_len', 'Swear_count', 'Fake_swear_count', 'Opinion_count', 'Prep_per_sent',
                     'Donation_appeal', 'Social']]

In [98]:
num_df.to_csv('data/num_df.pk')

In [99]:
target_df = podcast_df[['Hosts', 'Genre-Topic', 'Scripted/Un', 'Fiction/Non', 'Format', 'Rating', 'Year']]
target_df

,Hosts,Genre-Topic,Scripted/Un,Fiction/Non,Format,Rating,Year
podcast,,,,,,,
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,NaN
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,NaN
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,NaN
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,NaN
Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,NaN
...,...,...,...,...,...,...,...
You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,2021
You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,2021
You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,2021


In [100]:
target_df['Tag1'] = target_df['Genre-Topic'].map(lambda x: x[0])
target_df['Tag2'] = target_df['Genre-Topic'].map(lambda x: x[-1])

<ipython-input-100-7fa46ba2ed3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['Tag1'] = target_df['Genre-Topic'].map(lambda x: x[0])
<ipython-input-100-7fa46ba2ed3c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['Tag2'] = target_df['Genre-Topic'].map(lambda x: x[-1])


In [101]:
target_df = target_df.drop(['Genre-Topic'], axis=1)

In [102]:
target_num_df = pd.get_dummies(target_df, columns=['Scripted/Un', 'Fiction/Non', 'Format', 'Tag1', 'Tag2'])

In [103]:
target_df.to_csv('data/target_df.csv', encoding='utf-8')
target_num_df.to_csv('data/target_num_df.csv', encoding='utf-8')

In [104]:
podtoks_df = podcast_df[['Title', 'Tokens', 'Top50']]
podtoks_df

,Title,Tokens,Top50
podcast,,,
Bullseye with Jesse Thorn,Cartoonist and Author Adrian Tomine,"(jesse, thorn, Hey, all, ,, Jesse, ., As, 2020...","{for, about, brian, or, the, i, at, so, as, be..."
Bullseye with Jesse Thorn,"Cristin Milioti on ‘Palm Springs,’ ‘How I Met...","(music, Gentle, ,, trilling, music, with, a, s...","{for, lot, about, when, the, i, at, so, yeah, ..."
Bullseye with Jesse Thorn,Musician Frank Turner,"(music, Gentle, ,, trilling, music, with, a, s...","{for, about, or, the, i, we, so, at, nofx, as,..."
Bullseye with Jesse Thorn,Isiah Whitlock Jr.,"(music, Gentle, ,, trilling, music, with, a, s...","{for, about, when, or, the, i, at, so, be, do,..."
Bullseye with Jesse Thorn,"Maya Erskine and Anna Konkle of PEN15, back f...","(music, Gentle, ,, trilling, music, with, a, s...","{maya, for, about, the, i, at, so, yeah, we, a..."
...,...,...,...
You're Wrong About,The O.J. Simpson Trial: The DeLorean Detour,"(Sarah, :, Yeah, ., Instead, of, MLMs, in, the...","{for, about, cocaine, him, the, i, we, so, at,..."
You're Wrong About,Vanessa Williams Part 1: Becoming Miss America,"(Sarah, Marshall, :, Oh, ,, my, God, ., It, 's...","{for, michael, about, the, i, at, so, we, amer..."
You're Wrong About,"Bonus: ""The Dark Knight""","(Mike, :, Ooh, ,, I, have, one, ,, I, have, on...","{for, about, batman, aubrey, or, the, movie, i..."


In [105]:
podtoks_df.to_csv('data/podtoks_df.csv', encoding='utf-8')

In [106]:
pattern_maker()

enter tag (lowercase):  0


In [107]:
podcast_df['i_count'].to_csv('data/i_count.csv', encoding='utf-8')

In [119]:
one_host_pods = ['Welcome to Nightvale','Move Your DNA', 'You\'re Wrong About','Radiolab',
            'The Flophouse','MBMBaM','Sawbones','Wonderful','The Greatest Generation',
            'Friendly Fire','Shmanners','The Adventure Zone','Neoscum','The Allusionist']

In [124]:
# df.loc[df['column_name'] == some_value]
podcast_df = podcast_df.reset_index(drop=False)
podcast_df

,podcast,Hosts,Genre-Topic,Scripted/Un,Fiction/Non,Format,Rating,Episode,Host_speech,Text,...,Prep_per_sent,Donation_appeal,Social,Know,Be,Do,Mean,Make,Go,Avg_sent_len
0,Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,,NaN,"jesse thorn Hey all, Jesse. As 2020 draws to a...",...,1.733894,0,0,9.541,8.213,4.348,2.174,2.536,2.174,14.932773
1,Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,,NaN,"music Gentle, trilling music with a steady dru...",...,1.132492,0,0,9.266,9.965,5.682,2.535,0.000,1.573,11.175079
2,Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,,NaN,"music Gentle, trilling music with a steady dru...",...,1.839713,0,0,7.395,9.101,4.551,2.048,0.000,2.617,15.244019
3,Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,,NaN,"music Gentle, trilling music with a steady dru...",...,1.229008,0,0,7.600,7.322,5.561,2.502,2.873,4.449,12.150763
4,Bullseye with Jesse Thorn,1.5,"[comedy, society]",unscripted,nonfiction,interview,4.7,,NaN,"music Gentle, trilling music with a steady dru...",...,0.890719,0,0,5.010,7.620,3.027,1.670,1.775,2.818,8.965569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,,NaN,Sarah: Yeah. Instead of MLMs in the early eigh...,...,1.500867,0,0,3.980,13.516,2.488,0.000,2.322,4.726,14.632582
1677,You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,,NaN,"Sarah Marshall: Oh, my God. It's like a cat or...",...,1.142966,0,0,4.770,16.051,3.526,1.078,0.912,4.770,12.698859
1678,You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,,NaN,"Mike : Ooh, I have one, I have one I have one!...",...,1.458045,0,0,4.483,15.357,4.261,1.243,1.864,3.506,14.676209
1679,You're Wrong About,2.0,"[history, education]",unscripted,nonfiction,chat,4.6,,NaN,Sarah: I think that if we were going to have a...,...,1.542923,0,0,3.748,12.947,3.464,1.420,1.249,3.748,14.491879


In [127]:
# df.loc[df['column_name'] == some_value]
same_hosts = podcast_df[podcast_df['podcast'].isin(one_host_pods)]

In [131]:
same_hosts.columns

Index(['podcast', 'Hosts', 'Genre-Topic', 'Scripted/Un', 'Fiction/Non',
       'Format', 'Rating', 'Episode', 'Host_speech', 'Text', 'Title', 'Year',
       'Top50', 'Token_count', 'Token_lengths', 'Avg_token_len', 'TTR',
       'kband', 'Avg_kband', 'Bigrams', 'Bigram_top25', 'POS', 'POS_freq',
       'Noun_freq', 'Proper_noun_freq', 'Verb_freq', 'Adj_freq', 'Adv_freq',
       'Interjection_freq', 'Preposition_freq', 'Conjunction_freq',
       'POS_length', 'Avg_noun_len', 'Avg_verb_len', 'Avg_adj_len',
       'Avg_adv_len', 'Pron_counts', 'i_count', 'you_count', 'she_count',
       'he_count', 'it_count', 'they_count', 'we_count', 'verb_lemmas',
       'Sent_toks', 'Sent_length', 'Ents', 'Organization', 'Art', 'Date',
       'Geopolitical', 'Numbers', 'Event', 'Cash', 'Time', 'Product',
       'Punctuation', 'period_freq', 'excl_freq', 'quest_freq', 'hyph_freq',
       'Tags', 'Tag_len', 'Tag_top_verb', 'Swear_count', 'Fake_swear_count',
       'Opinion_count', 'Prep_per_sent', 'Dona

Radiolab                   192
Welcome to Nightvale       168
Move Your DNA              104
The Allusionist             97
MBMBaM                      32
Sawbones                    30
Wonderful                   29
Shmanners                   28
The Greatest Generation     28
Friendly Fire               28
The Adventure Zone          19
The Flophouse               16
You're Wrong About          13
Name: podcast, dtype: int64

TypeError: reset_index() got an unexpected keyword argument 'columns'